# Project Topic:Ranking of Academic Papers


## Loading the DBLP-Citation-network V10 data into dataframes

In [2]:
import os
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize

import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
def load_df(json_path='./project/dblp-ref-1.json', nrows=None):
    train = pd.read_json(json_path, lines=True)
    train.reset_index(level=0, inplace=True)
    train.head()
    return train

In [3]:
train_df = load_df()
train_df.head()
# len(train_df)
# dict_paper_refs = {}

,index,abstract,authors,id,n_citation,references,title,venue,year
0,0,Based on biological control strategy in pest m...,"[Guoping Pang, Lansun Chen]",4aa69add-3978-480b-a1c0-d99a83d7e324,8,"[04754a28-6bf4-4d5d-8e42-2677d8564cdc, 33a877a...",Dynamic analysis of a pest-epidemic model with...,Mathematics and Computers in Simulation,2008
1,1,Abstract#R##N##R##N#Regression testing is an i...,"[Anneliese von Mayrhauser, Ning Zhang]",4aa69b49-9bfc-49c3-81a7-7af5c215ca0e,50,"[2932090c-8204-4970-bd27-98a66155011c, 9479c8d...",Automated regression testing using DBT and Sleuth,Journal of Software Maintenance and Evolution:...,1999
2,2,Mobile applications are software packages that...,"[Zheng Yan, Peng Zhang, Robert H. Deng]",4aa69de5-c819-482e-a808-2b3f8b829008,53,"[012d9aaf-34d0-496a-99c0-f0fd15dd2984, 029be9a...",TruBeRepec: a trust-behavior-based reputation ...,ubiquitous computing,2012
3,3,We propose a method that allows opportunistic ...,"[Jarkko Kaleva, Petri Komulainen, Markku J. Ju...",4aa6a255-c72c-4ccf-b1b8-c048ba39255b,0,"[1322cb89-dda2-4625-8c80-6863810d5c3f, 1eddb4e...",Zero-forcing spatial interweave with greedy sc...,wireless personal multimedia communications,2011
4,4,This article studies the power allocation (PA)...,"[Yingnan Zhang, Qiao Wang]",4aa6bae5-bbd9-46a1-944a-18ed700b72d5,50,"[0b480a7b-b171-4bf6-846c-1c7478a6d279, 0fd4be7...",Optimal power control of parallel orthogonalfr...,Iet Communications,2011


In [1]:
train_df.describe()
# train_df.to_csv('dblp-ref-1_1M.csv',index = False, header=False)
# venues= train_df.venue.unique()
# venues[:100]

In [17]:
import requests
import re
def get_field_of_study(df_titles=None):
    print(len(df_titles))
    df_titles_copy = df_titles.copy()
#     titles = ['temporal languages for simulation and analysis of the dynamics within an organisation','on leveraging executable language engineering for domain specific transformation languages'] 
#     payload = {'expr': 'Or(Ti==\'on leveraging executable language engineering for domain specific transformation languages\', Ti=\'temporal languages for simulation and analysis of the dynamics within an organisation\')', 'attributes': 'Ti,AA.AuN,AA.AuId,F.FN'}
#     headers = {'Ocp-Apim-Subscription-Key': '78488b64dc7541f1a80ba499b9c5f8ff'}
#     req = requests.get('https://api.labs.cognitive.microsoft.com/academic/v1.0/evaluate', params=payload, headers=headers)
#     response = req.json()
    field1=[]
    field2=[]
    field3=[]
    index = 0
    call_count=0
    while index<len(df_titles):#len(df_titles)
        titles = []
        if index +10 < len(df_titles):
            titles=df_titles[index:index+10]
#             titles = titles.reset_index()
#           print(len(titles))
        else:
            titles=df_titles[index:]
#         print(titles[0])
#         print(str(titles.iloc[1]))
    #     payload = {'expr': 'Or(Ti==\'on leveraging executable language engineering for domain specific transformation languages\', Ti=\'temporal languages for simulation and analysis of the dynamics within an organisation\')', 'attributes': 'Ti,AA.AuN,AA.AuId,F.FN'}
        titles_str = ''

        for j in range(0, len(titles)):#len(titles)
            titles.iloc[j] = titles.iloc[j].replace("-"," ")
            titles.iloc[j] = titles.iloc[j].replace(":"," ")
            titles.iloc[j] = titles.iloc[j].replace("/"," ")
            titles.iloc[j] = titles.iloc[j].replace(".","")
            titles.iloc[j] = str(titles.iloc[j]).lower()
            titles.iloc[j] = re.sub('[^a-z0-9 \.]', '', titles.iloc[j])
            titles.iloc[j] = re.sub(' +',' ',titles.iloc[j])
            if j != len(titles)-1:
                titles_str=titles_str+'Ti==\''+str(titles.iloc[j])+'\','
            else:
                titles_str=titles_str+'Ti==\''+str(titles.iloc[j])+'\''
        payload = {'expr': 'Or('+titles_str+')', 'attributes': 'Ti,AA.AuN,AA.AuId,F.FN'}
#         print (payload)
        
        keys = ['5f6964f3e4ee460b83a3d0c32756b011','9fecb44f57874a1f9272dc36ffa38276','948b52f75c714d61be3a81037c8772ce','05c5287ad30c4daab18895ba4593ba9d']
        k = call_count%len(keys)
#         print ('using key at index:'+str(k)+'value'+str(keys[k]) + 'index='+str(index))
        headers = {'Ocp-Apim-Subscription-Key': keys[k]}
        import time
        if  call_count%20==0:
            print ('sleeping')
            time.sleep(.300)
            time.sleep(.300)
            time.sleep(.100)
        req = requests.get('https://api.labs.cognitive.microsoft.com/academic/v1.0/evaluate', params=payload, headers=headers)
        if req.status_code != 200:
            print(index) 
            print ("FAILED WITH STATUS"+ str(req.status_code))
        response = req.json()
        call_count = call_count+1
        
        for j in range(0, len(titles)):
#             print('TITLE: ',titles.iloc[j])
            if response is not None:
                if response.get('entities') is not None:

                    for i in range(0,len(response.get('entities'))):
                        if response.get('entities')[i].get('Ti')!=titles.iloc[j]:
                            continue
                        titles.iloc[j]='visited'
                        if response.get('entities')[i].get('F') is not None:
                            fields = response.get('entities')[i].get('F')
#                             print(fields)
#                             print(len(fields))
                            fields_size = len(fields)
                    #         for i in range(len(fields-1, len(fields)-4,-1 ):
                            if fields_size > 2:
                                field1.append(fields[fields_size-1].get('FN'))
                                field2.append(fields[fields_size-2].get('FN'))
                                field3.append(fields[fields_size-3].get('FN'))

                            elif fields_size == 2:
                                field1.append(fields[fields_size-1].get('FN'))
                                field2.append(fields[fields_size-2].get('FN'))
                                field3.append('NA')
                            elif fields_size == 1:
                                field1.append(fields[fields_size-1].get('FN'))
                                field2.append('NA')
                                field3.append('NA')
                            else:
                                field1.append('NA')
                                field2.append('NA')
                                field3.append('NA')

            if len(field1)-1<index:
                field1.append('NA')
            if len(field2)-1<index:
                field2.append('NA')
            if len(field3)-1<index:
                field3.append('NA')
#             print(field3[index]+' '+field2[index]+' '+field1[index])
            index=index+1
#         print(len(field1))
#         print(len(field2))
#         print(len(field3))
        if (index%100==0 and index>0):
            print('write to file')
            title_field = pd.DataFrame(
            {'title': df_titles_copy[index-100:index], 'field1': field1[index-100:index], 'field2': field2[index-100:index], 'field3': field3[index-100:index]},
            columns = ['title', 'field1', 'field2','field3'])
            title_field.to_csv('fields_dblp-ref-1_from600k.csv', mode='a',index = False, header=False)
            print('Completed: '+str(index))
    if (index%100!=0 and index>0):
        print('last write to file')
        title_field = pd.DataFrame(
        {'title': df_titles_copy[index-(index%100):index], 'field1': field1[index-(index%100):index], 'field2': field2[index-(index%100):index], 'field3': field3[index-(index%100):index]},
        columns = ['title', 'field1', 'field2','field3'])
        title_field.to_csv('fields_dblp-ref-1_from600k.csv', mode='a',index = False, header=False)

In [24]:
df_titles = train_df.title.copy()
get_field_of_study(df_titles[:10000])

200000
sleeping
write to file
Completed: 100
write to file
Completed: 200
sleeping
write to file
Completed: 300
write to file
Completed: 400
sleeping
write to file
Completed: 500
write to file
Completed: 600
sleeping
write to file
Completed: 700
write to file
Completed: 800
sleeping
write to file
Completed: 900
write to file
Completed: 1000
sleeping
write to file
Completed: 1100
write to file
Completed: 1200
sleeping
write to file
Completed: 1300
write to file
Completed: 1400
sleeping
write to file
Completed: 1500
write to file
Completed: 1600
sleeping
write to file
Completed: 1700
write to file
Completed: 1800
sleeping
write to file
Completed: 1900
write to file
Completed: 2000
sleeping
write to file
Completed: 2100
write to file
Completed: 2200
sleeping
write to file
Completed: 2300
write to file
Completed: 2400
sleeping
write to file
Completed: 2500
write to file
Completed: 2600
sleeping
write to file
Completed: 2700
write to file
Completed: 2800
sleeping
write to file
Completed: 29

write to file
Completed: 23500
write to file
Completed: 23600
sleeping
write to file
Completed: 23700
write to file
Completed: 23800
sleeping
write to file
Completed: 23900
write to file
Completed: 24000
sleeping
write to file
Completed: 24100
write to file
Completed: 24200
sleeping
write to file
Completed: 24300
write to file
Completed: 24400
sleeping
write to file
Completed: 24500
write to file
Completed: 24600
sleeping
write to file
Completed: 24700
write to file
Completed: 24800
sleeping
write to file
Completed: 24900
write to file
Completed: 25000
sleeping
write to file
Completed: 25100
write to file
Completed: 25200
sleeping
write to file
Completed: 25300
write to file
Completed: 25400
sleeping
write to file
Completed: 25500
write to file
Completed: 25600
sleeping
write to file
Completed: 25700
write to file
Completed: 25800
sleeping
write to file
Completed: 25900
write to file
Completed: 26000
sleeping
write to file
Completed: 26100
write to file
Completed: 26200
sleeping
write 

write to file
Completed: 46600
sleeping
write to file
Completed: 46700
write to file
Completed: 46800
sleeping
write to file
Completed: 46900
write to file
Completed: 47000
sleeping
write to file
Completed: 47100
write to file
Completed: 47200
sleeping
write to file
Completed: 47300
write to file
Completed: 47400
sleeping
write to file
Completed: 47500
write to file
Completed: 47600
sleeping
write to file
Completed: 47700
write to file
Completed: 47800
sleeping
write to file
Completed: 47900
write to file
Completed: 48000
sleeping
write to file
Completed: 48100
write to file
Completed: 48200
sleeping
write to file
Completed: 48300
write to file
Completed: 48400
sleeping
write to file
Completed: 48500
write to file
Completed: 48600
sleeping
write to file
Completed: 48700
write to file
Completed: 48800
sleeping
write to file
Completed: 48900
write to file
Completed: 49000
sleeping
write to file
Completed: 49100
write to file
Completed: 49200
sleeping
write to file
Completed: 49300
write 

write to file
Completed: 69700
write to file
Completed: 69800
sleeping
write to file
Completed: 69900
write to file
Completed: 70000
sleeping
write to file
Completed: 70100
write to file
Completed: 70200
sleeping
write to file
Completed: 70300
write to file
Completed: 70400
sleeping
write to file
Completed: 70500
write to file
Completed: 70600
sleeping
write to file
Completed: 70700
write to file
Completed: 70800
sleeping
write to file
Completed: 70900
write to file
Completed: 71000
sleeping
write to file
Completed: 71100
write to file
Completed: 71200
sleeping
write to file
Completed: 71300
write to file
Completed: 71400
sleeping
write to file
Completed: 71500
write to file
Completed: 71600
sleeping
write to file
Completed: 71700
write to file
Completed: 71800
sleeping
write to file
Completed: 71900
write to file
Completed: 72000
sleeping
write to file
Completed: 72100
write to file
Completed: 72200
sleeping
write to file
Completed: 72300
write to file
Completed: 72400
sleeping
write 

write to file
Completed: 92800
sleeping
write to file
Completed: 92900
write to file
Completed: 93000
sleeping
write to file
Completed: 93100
write to file
Completed: 93200
sleeping
write to file
Completed: 93300
write to file
Completed: 93400
sleeping
write to file
Completed: 93500
write to file
Completed: 93600
sleeping
write to file
Completed: 93700
write to file
Completed: 93800
sleeping
write to file
Completed: 93900
write to file
Completed: 94000
sleeping
write to file
Completed: 94100
write to file
Completed: 94200
sleeping
write to file
Completed: 94300
write to file
Completed: 94400
sleeping
write to file
Completed: 94500
write to file
Completed: 94600
sleeping
write to file
Completed: 94700
write to file
Completed: 94800
sleeping
write to file
Completed: 94900
write to file
Completed: 95000
sleeping
write to file
Completed: 95100
write to file
Completed: 95200
sleeping
write to file
Completed: 95300
write to file
Completed: 95400
sleeping
write to file
Completed: 95500
write 

write to file
Completed: 115500
write to file
Completed: 115600
sleeping
write to file
Completed: 115700
write to file
Completed: 115800
sleeping
write to file
Completed: 115900
write to file
Completed: 116000
sleeping
write to file
Completed: 116100
write to file
Completed: 116200
sleeping
write to file
Completed: 116300
write to file
Completed: 116400
sleeping
write to file
Completed: 116500
write to file
Completed: 116600
sleeping
write to file
Completed: 116700
write to file
Completed: 116800
sleeping
write to file
Completed: 116900
write to file
Completed: 117000
sleeping
write to file
Completed: 117100
write to file
Completed: 117200
sleeping
write to file
Completed: 117300
write to file
Completed: 117400
sleeping
write to file
Completed: 117500
write to file
Completed: 117600
sleeping
write to file
Completed: 117700
write to file
Completed: 117800
sleeping
write to file
Completed: 117900
write to file
Completed: 118000
sleeping
write to file
Completed: 118100
write to file
Compl

write to file
Completed: 138000
sleeping
write to file
Completed: 138100
write to file
Completed: 138200
sleeping
write to file
Completed: 138300
write to file
Completed: 138400
sleeping
write to file
Completed: 138500
write to file
Completed: 138600
sleeping
write to file
Completed: 138700
write to file
Completed: 138800
sleeping
write to file
Completed: 138900
write to file
Completed: 139000
sleeping
write to file
Completed: 139100
write to file
Completed: 139200
sleeping
write to file
Completed: 139300
write to file
Completed: 139400
sleeping
write to file
Completed: 139500
write to file
Completed: 139600
sleeping
write to file
Completed: 139700
write to file
Completed: 139800
sleeping
write to file
Completed: 139900
write to file
Completed: 140000
sleeping
write to file
Completed: 140100
write to file
Completed: 140200
sleeping
write to file
Completed: 140300
write to file
Completed: 140400
sleeping
write to file
Completed: 140500
write to file
Completed: 140600
sleeping
write to f

write to file
Completed: 160500
write to file
Completed: 160600
sleeping
write to file
Completed: 160700
write to file
Completed: 160800
sleeping
write to file
Completed: 160900
write to file
Completed: 161000
sleeping
write to file
Completed: 161100
write to file
Completed: 161200
sleeping
write to file
Completed: 161300
write to file
Completed: 161400
sleeping
write to file
Completed: 161500
write to file
Completed: 161600
sleeping
write to file
Completed: 161700
write to file
Completed: 161800
sleeping
write to file
Completed: 161900
write to file
Completed: 162000
sleeping
write to file
Completed: 162100
write to file
Completed: 162200
sleeping
write to file
Completed: 162300
write to file
Completed: 162400
sleeping
write to file
Completed: 162500
write to file
Completed: 162600
sleeping
write to file
Completed: 162700
write to file
Completed: 162800
sleeping
write to file
Completed: 162900
write to file
Completed: 163000
sleeping
write to file
Completed: 163100
write to file
Compl

write to file
Completed: 183000
sleeping
write to file
Completed: 183100
write to file
Completed: 183200
sleeping
write to file
Completed: 183300
write to file
Completed: 183400
sleeping
write to file
Completed: 183500
write to file
Completed: 183600
sleeping
write to file
Completed: 183700
write to file
Completed: 183800
sleeping
write to file
Completed: 183900
write to file
Completed: 184000
sleeping
write to file
Completed: 184100
write to file
Completed: 184200
sleeping
write to file
Completed: 184300
write to file
Completed: 184400
sleeping
write to file
Completed: 184500
write to file
Completed: 184600
sleeping
write to file
Completed: 184700
write to file
Completed: 184800
sleeping
write to file
Completed: 184900
write to file
Completed: 185000
sleeping
write to file
Completed: 185100
write to file
Completed: 185200
sleeping
write to file
Completed: 185300
write to file
Completed: 185400
sleeping
write to file
Completed: 185500
write to file
Completed: 185600
sleeping
write to f

In [4]:
# len(df_titles)
# print (train_df.title[0:1007].tail(10))
data = pd.read_csv("fields_dblp-ref-1.csv", names=['Title', 'F1', "F2",'F3']) 
print(len(data))
# data.tail(10)
# data.Title
set(train_df.title).difference(set(data.Title))
# print (len(set(data.Title)))
# print (len(set(train_df.title[0:1007])))

1000000


set()

In [33]:
print(train_df.title[799999:800002])
print(len(train_df.title[0:]))
rows_relevant = train_df.loc[(train_df['title'] == 'Epidemiography with various growth functions')]

799999    Power-efficient clustering via incomplete bypa...
800000    On processing nested queries in distributed ob...
800001    Plane-Sweep Incremental Algorithm: Computing D...
Name: title, dtype: object
1000000


In [7]:
print(len(train_df))
# field_data = pd.read_csv("fields_dblp-ref-1.csv", names=['Title', 'F1', "F2",'F3']) 
print(len(data))
train_df['F1']=data.F1
train_df['F2']=data.F2
train_df['F3']=data.F3
train_df.to_csv('dblp-ref-1_full.csv',index = False, header=True)

1000000
1000000
